# Use Resnet to classify Cifar-10

In [0]:
!pip install torch torchvision

## Introduction

This project is to build a ResNet and use it to classify Cifar-10 dataset.
The ResNet architecture is introduced in 2015 by Kaiming He, et al. in the paper ["Deep residual learning for image recognition"](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf).

## Implement ResNet

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Conv2D(nn.Module):
    
    def __init__(self, inchannel, outchannel, kernel_size, stride, padding, bias = True):
        
        super(Conv2D, self).__init__()
        
        self.inchannel = inchannel
        self.outchannel = outchannel
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        self.weights = nn.Parameter(torch.Tensor(outchannel, inchannel, 
                                                 kernel_size, kernel_size))
        #self.weights.data.normal_(-0.1, 0.1)
        self.weights = nn.init.kaiming_normal_(self.weights)
        
        if bias:
            self.bias = nn.Parameter(torch.Tensor(outchannel, ))
            self.bias.data.normal_(-0.1, 0.1)
        else:
            self.bias = None
            
        
    def forward(self, x):
        
        unfold = nn.Unfold(kernel_size = (self.kernel_size, self.kernel_size), padding = self.padding, stride = self.stride).to('cuda')
        inp_unf = unfold(x).to('cuda')
        out_unf = inp_unf.transpose(1,2).matmul(self.weights.view(self.weights.size(0), -1).t()).transpose(1,2).to('cuda')
        x_out = (x.size(2)+2*self.padding - self.kernel_size)//self.stride + 1
        y_out = (x.size(3)+2*self.padding - self.kernel_size)//self.stride + 1
        output = F.fold(out_unf, (x_out, y_out), (1, 1)).to('cuda')        

        return output
        

In [0]:
class MaxPool2D(nn.Module):
    
    def __init__(self, pooling_size):
        # assume pooling_size = kernel_size = stride
        
        super(MaxPool2D, self).__init__()
        
        self.pooling_size = pooling_size
        

    def forward(self, x):
        
        #output = torch.zeros(x.size(0),x.size(1),x.size(2)//self.pooling_size, x.size(3)//self.pooling_size).to('cuda')
        #for i in range(0,x.size(2),self.pooling_size):
        #    for j in range(0,x.size(3),self.pooling_size):
        #        t = x[:,:,i:i+self.pooling_size,j:j+self.pooling_size].to('cuda')
        #        t = t.flatten(start_dim = 2).to('cuda')
        #        t = t.max(dim =2)
        #        output[:,:,i//self.pooling_size, j//self.pooling_size] = t[0].to('cuda')
        
        ### without loop method
        N, C_in, H_in, W_in = x.shape
        
        x_reshaped = x.reshape([N, C_in, H_in // self.pooling_size, self.pooling_size, 
                           W_in // self.pooling_size, self.pooling_size])
        
        out, _ = x_reshaped.max(3)
        
        output, _ = out.max(4)
                
        return output
        

In [0]:
# define resnet building blocks

class ResidualBlock(nn.Module): 
    def __init__(self, inchannel, outchannel, stride=1): 
        
        super(ResidualBlock, self).__init__() 
        
        self.left = nn.Sequential(Conv2D(inchannel, outchannel, kernel_size=3, 
                                         stride=stride, padding=1, bias=False), 
                                  nn.BatchNorm2d(outchannel), 
                                  nn.ReLU(inplace=True), 
                                  Conv2D(outchannel, outchannel, kernel_size=3, 
                                         stride=1, padding=1, bias=False), 
                                  nn.BatchNorm2d(outchannel)) 
        
        self.shortcut = nn.Sequential() 
        
        if stride != 1 or inchannel != outchannel: 
            
            self.shortcut = nn.Sequential(Conv2D(inchannel, outchannel, 
                                                 kernel_size=1, stride=stride, 
                                                 padding = 0, bias=False), 
                                          nn.BatchNorm2d(outchannel) ) 
            
    def forward(self, x): 
        
        out = self.left(x) 
        
        out += self.shortcut(x) 
        
        out = F.relu(out) 
        
        return out


In [0]:
# define resnet

class ResNet(nn.Module):
    
    def __init__(self, ResidualBlock, num_classes = 10):
        
        super(ResNet, self).__init__()
        
        self.inchannel = 64
        self.conv1 = nn.Sequential(Conv2D(3, 64, kernel_size = 3, stride = 1,
                                            padding = 1, bias = False), 
                                  nn.BatchNorm2d(64), 
                                  nn.ReLU())
        
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride = 1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride = 2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride = 2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride = 2)
        self.maxpool = MaxPool2D(4)
        self.fc = nn.Linear(512, num_classes)
        
    
    def make_layer(self, block, channels, num_blocks, stride):
        
        strides = [stride] + [1] * (num_blocks - 1)
        
        layers = []
        
        for stride in strides:
            
            layers.append(block(self.inchannel, channels, stride))
            
            self.inchannel = channels
            
        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        
        x = self.conv1(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.maxpool(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc(x)
        
        return x
    
    
def ResNet18():
    return ResNet(ResidualBlock)

## Train ResNet

In [0]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset

import numpy as np

import torchvision.transforms as T


transform = T.ToTensor()


# load data

NUM_TRAIN = 49000
print_every = 100


data_dir = './data'
cifar10_train = dset.CIFAR10(data_dir, train=True, download=True, transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10(data_dir, train=True, download=True, transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10(data_dir, train=False, download=True, transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)


USE_GPU = True
dtype = torch.float32 

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

Files already downloaded and verified
Files already downloaded and verified


In [0]:
def check_accuracy(loader, model):
    # function for test accuracy on validation and test set
    
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))


def train_part(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        print(len(loader_train))
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            loss.backward()

            # Update the parameters of the model using the gradients
            optimizer.step()

            if t % print_every == 0:
                print('Epoch: %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                #check_accuracy(loader_val, model)
                print()
        

In [0]:
# code for optimising network performance

del cifar10_train
del loader_train
del cifar10_val
del loader_val
del cifar10_test
del loader_test


data_transforms = T.Compose([
        T.ToTensor(),
        #T.Normalize([0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]),
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])


cifar10_train = dset.CIFAR10(data_dir, train=True, download=True, transform=data_transforms)
loader_train = DataLoader(cifar10_train, batch_size=64,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10(data_dir, train=True, download=True, transform=data_transforms)
loader_val = DataLoader(cifar10_val, batch_size=64,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10(data_dir, train=False, download=True, transform=data_transforms)
loader_test = DataLoader(cifar10_test, batch_size=64)


USE_GPU = True
dtype = torch.float32 

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
    
   
#!pip install GPy
#!pip install GPyOpt
#import GPy
#import GPyOpt

#from GPyOpt.methods import BayesianOptimization


#def cv_score(parameters):
#  parameters=parameters[0]
#  lr=parameters[0]
#  model = ResNet18()
#  optimizer = optim.Adam(model.parameters(),lr=lr)
#  train_part(model, optimizer, epochs = 5)
#  acc=check_accuracy(loader_test, model)
#  score=np.array(acc)
#  return score

#bds = [{'name': 'learning_rate', 'type': 'continuous', 'domain': (0.0001, 0.01)}]
#optimizer = BayesianOptimization(f=cv_score, 
#                                 domain=bds,
#                                 model_type='GP',
#                                 acquisition_type ='EI',
#                                 acquisition_jitter = 0.05,
#                                 exact_feval=True, 
#                                 maximize=True)
#optimizer.run_optimization(max_iter=10)
#optimizer.plot_acquisition()
#opt = optimizer.x_opt


# define and train the network
model = ResNet18()
optimizer = optim.Adam(model.parameters(), lr= opt[0]) #for bayesian optimization, please uncomment the code above


train_part(model, optimizer, epochs = 5)


# report test set accuracy

check_accuracy(loader_test, model)


# save the model
torch.save(model.state_dict(), 'model.pt')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
766
Epoch: 0, Iteration 0, loss = 2.9305

Epoch: 0, Iteration 100, loss = 1.5960

Epoch: 0, Iteration 200, loss = 1.5995

Epoch: 0, Iteration 300, loss = 1.0909

Epoch: 0, Iteration 400, loss = 1.1119

Epoch: 0, Iteration 500, loss = 1.4162

Epoch: 0, Iteration 600, loss = 0.7753

Epoch: 0, Iteration 700, loss = 1.0101

766
Epoch: 1, Iteration 0, loss = 0.6692

Epoch: 1, Iteration 100, loss = 0.7235

Epoch: 1, Iteration 200, loss = 0.7129

Epoch: 1, Iteration 300, loss = 0.6694

Epoch: 1, Iteration 400, loss = 0.7546

Epoch: 1, Iteration 500, loss = 0.5350

Epoch: 1, Iteration 600, loss = 0.6856

Epoch: 1, Iteration 700, loss = 0.5922

766
Epoch: 2, Iteration 0, loss = 0.4598

Epoch: 2, Iteration 100, loss = 0.3009

Epoch: 2, Iteration 200, loss = 0.4092

Epoch: 2, Iteration 300, loss = 0.5850

Epoch: 2, Iteration 400, loss = 0.4897

Epoch: 2, Iteration 500, loss = 0.4609

